In [2]:
import pm4py
import pandas as pd
import networkx as nx

ocel = pm4py.read_ocel2_sqlite("../data/event_logs/ContainerLogistics-v2.sqlite")
big_ocel = pm4py.read_ocel2_sqlite("/home/grkmr/Downloads/order-management(3).sqlite")

/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:75: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_23_foreign_key_event_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/util/dataframe_utils.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pandas_utils.dataframe_column_string_to_datetime(df[col], format=timest_format, utc=True)
/home/grkmr/Projects/ocelescope/src/backend/.venv/lib/python3.10/site-packages/pm4py/objects/log/u

In [4]:
ocdf = pm4py.discover_ocdfg(ocel)

In [5]:
ocdf.keys()

dict_keys(['activities', 'object_types', 'edges', 'activities_indep', 'activities_ot', 'start_activities', 'end_activities', 'edges_performance'])

In [38]:
for object_type,edge in ocdf["edges"]["event_couples"].items():
    print(object_type, list(edge.keys()),"\n")

Customer Order [('Register Customer Order', 'Create Transport Document')] 

Transport Document [('Create Transport Document', 'Book Vehicles'), ('Book Vehicles', 'Order Empty Containers'), ('Order Empty Containers', 'Depart'), ('Depart', 'Depart'), ('Depart', 'Reschedule Container'), ('Reschedule Container', 'Depart'), ('Order Empty Containers', 'Reschedule Container'), ('Reschedule Container', 'Reschedule Container')] 

Container [('Order Empty Containers', 'Pick Up Empty Container'), ('Pick Up Empty Container', 'Load Truck'), ('Load Truck', 'Load Truck'), ('Load Truck', 'Drive to Terminal'), ('Drive to Terminal', 'Weigh'), ('Weigh', 'Place in Stock'), ('Place in Stock', 'Bring to Loading Bay'), ('Bring to Loading Bay', 'Load to Vehicle'), ('Weigh', 'Bring to Loading Bay'), ('Load to Vehicle', 'Depart'), ('Bring to Loading Bay', 'Reschedule Container'), ('Reschedule Container', 'Load to Vehicle')] 

Vehicle [('Book Vehicles', 'Book Vehicles'), ('Book Vehicles', 'Load to Vehicle'), ('L

In [25]:
for object_type, start_activity in ocdf["start_activities"]["events"].items():
    print(object_type, start_activity.keys(), "/n")

Container dict_keys(['Order Empty Containers']) /n
Customer Order dict_keys(['Register Customer Order']) /n
Transport Document dict_keys(['Create Transport Document']) /n
Vehicle dict_keys(['Book Vehicles']) /n


In [26]:
for object_type, start_activity in ocdf["end_activities"]["events"].items():
    print(object_type, start_activity.keys(), "/n")

Container dict_keys(['Bring to Loading Bay', 'Depart', 'Load Truck', 'Load to Vehicle', 'Order Empty Containers', 'Pick Up Empty Container', 'Place in Stock', 'Reschedule Container']) /n
Customer Order dict_keys(['Create Transport Document', 'Register Customer Order']) /n
Transport Document dict_keys(['Book Vehicles', 'Depart', 'Order Empty Containers', 'Reschedule Container']) /n
Vehicle dict_keys(['Depart']) /n


In [44]:
from pydantic import BaseModel

class DFG_EDGE(BaseModel):
    source: str
    target: str
    object_type:str


class OCDFG(BaseModel):
    object_types : list[str]
    activities: list[str]
    edges: list[DFG_EDGE]
    start_activities: dict[str,list[str]]
    end_activities: dict[str,list[str]]

In [54]:
def compute_ocdfg(ocel:pm4py.OCEL) -> OCDFG:
    ocdfg = pm4py.discover_ocdfg(ocel)

    edges = []
    for object_type,raw_edges in ocdf["edges"]["event_couples"].items():
        edges = edges + ([ DFG_EDGE(object_type=object_type,source=source, target=target) for source, target in raw_edges])

    start_activities ={object_type :list(raw_start_activities) for object_type, raw_start_activities in ocdf["start_activities"]["events"].items()}
    end_activities ={object_type :list(raw_start_activities) for object_type, raw_start_activities in ocdf["start_activities"]["events"].items()}
    
    return OCDFG(activities=ocdfg["activities"], edges=edges, object_types=ocdfg["object_types"],end_activities=end_activities, start_activities=start_activities )

compute_ocdfg(ocel)

OCDFG(object_types=['Customer Order', 'Vehicle', 'Container', 'Transport Document'], activities=['Drive to Terminal', 'Book Vehicles', 'Order Empty Containers', 'Collect Goods', 'Reschedule Container', 'Depart', 'Create Transport Document', 'Load to Vehicle', 'Load Truck', 'Weigh', 'Pick Up Empty Container', 'Bring to Loading Bay', 'Place in Stock', 'Register Customer Order'], edges=[DFG_EDGE(source='Register Customer Order', target='Create Transport Document', object_type='Customer Order'), DFG_EDGE(source='Create Transport Document', target='Book Vehicles', object_type='Transport Document'), DFG_EDGE(source='Book Vehicles', target='Order Empty Containers', object_type='Transport Document'), DFG_EDGE(source='Order Empty Containers', target='Depart', object_type='Transport Document'), DFG_EDGE(source='Depart', target='Depart', object_type='Transport Document'), DFG_EDGE(source='Depart', target='Reschedule Container', object_type='Transport Document'), DFG_EDGE(source='Reschedule Contai